In [621]:
import itertools
import pandas as pd

In [622]:
# Weights for base probability
# source at: https://genshin-impact.fandom.com/wiki/Artifact/Distribution#Sub_Stat_Attribute

substats_weights = lambda: {
    "hp": 6,
    "atk": 6,
    "def": 6,
    "hp_percent": 4,
    "atk_percent": 4,
    "def_percent": 4,
    "energy_recharge": 4,
    "elemental_mastery": 4,
    "crit_rate": 3,
    "crit_dmg": 3
}

substats_keys = lambda: substats_weights().keys()

In [623]:
# Computes base probabilities from available substats' weight.
# The "base" probability is the chance for a substat of being selected first in
# the artifact.
# - exclude: substats that are not available (e.g. a mainstat)
def get_substats_base_probabiliy(exclude=[]):
  total_weight = 0

  # Total available weight
  for stat, weight in substats_weights().items():
    if stat not in exclude:
      total_weight += weight

  substats_base_probability = {}
  # probability = weight / total_weight
  for stat, weight in substats_weights().items():
    if stat not in exclude:
      substats_base_probability[stat] = weight / total_weight
  return substats_base_probability

In [624]:
# HP mainstat
get_substats_base_probabiliy(["hp"])

{'atk': 0.15789473684210525,
 'def': 0.15789473684210525,
 'hp_percent': 0.10526315789473684,
 'atk_percent': 0.10526315789473684,
 'def_percent': 0.10526315789473684,
 'energy_recharge': 0.10526315789473684,
 'elemental_mastery': 0.10526315789473684,
 'crit_rate': 0.07894736842105263,
 'crit_dmg': 0.07894736842105263}

In [625]:
# ATK% mainstat
get_substats_base_probabiliy(["atk_percent"])

{'hp': 0.15,
 'atk': 0.15,
 'def': 0.15,
 'hp_percent': 0.1,
 'def_percent': 0.1,
 'energy_recharge': 0.1,
 'elemental_mastery': 0.1,
 'crit_rate': 0.075,
 'crit_dmg': 0.075}

In [626]:
# Elemental DMG mainstat
get_substats_base_probabiliy()

{'hp': 0.13636363636363635,
 'atk': 0.13636363636363635,
 'def': 0.13636363636363635,
 'hp_percent': 0.09090909090909091,
 'atk_percent': 0.09090909090909091,
 'def_percent': 0.09090909090909091,
 'energy_recharge': 0.09090909090909091,
 'elemental_mastery': 0.09090909090909091,
 'crit_rate': 0.06818181818181818,
 'crit_dmg': 0.06818181818181818}

In [627]:
# Generates all the possible combinations for the given substats to be selected
# in the artifact.
# - substats: which substats are expected, max 4
# - exclude: substats that are not available (e.g. a mainstat)
# 'substats' should not contain values from 'exclude'
def get_substats_combinations(substats=[], exclude=[]):
  positions = (0, 1, 2, 3)

  available_substats = []
  for stat in substats_keys():
    if stat not in exclude and stat not in substats:
      available_substats.append(stat)

  # All positions in which the expected substats can be selected
  expected_positions = itertools.permutations(positions, min(4, len(substats)))

  for used_pos in expected_positions:

    # Positions available for other substats
    available_pos = [0, 1, 2, 3]
    for pos in used_pos:
      available_pos.remove(pos)
    available_pos = tuple(available_pos)

    # All ways to fill the available space with other substats
    other_substats = itertools.permutations(available_substats, len(available_pos))

    # Fill all the 4 substat positions
    for other in other_substats:
      result = [0, 1, 2, 3]

      for i in range(len(used_pos)):
        result[used_pos[i]] = substats[i]

      for i in range(len(available_pos)):
        result[available_pos[i]] = other[i]

      yield result



In [628]:
# Computes the probability of getting a given combination of substats, in the
# specified order.
# - combination: the substats to get, placed in order
# - base_probabilities: map containing the chance of all substats to be selected
#   first.


def get_occurence_probability(combination=[], base_probabilities={}):
  total = 1
  previous = 0
  for i in range(len(combination)):

    # The probability of a substat to be selected changes according
    # to the substats that were selected previously

    base = base_probabilities[combination[i]]
    total *= base / (1 - previous)

    previous += base

  return total

# Run the program

In [629]:
# Computes the probability of getting a subset of substats in an artifact
def run(substats=[], exclude=[]):
  base_probabilities = get_substats_base_probabiliy(exclude)
  all_combinations = get_substats_combinations(substats, exclude)

  total = 0

  for combination in all_combinations:
    p = get_occurence_probability(combination, base_probabilities)
    total += p

  return total

In [630]:
# DMG Goblet with double crit
run(["crit_rate", "crit_dmg"])

0.06693227025946398

In [631]:
# Mainstat with ATK% and flat atk
run(["atk"], exclude=["atk_percent"])

0.5597652158461518

In [632]:
# Mainstat with ATK% and flat atk
run(["atk"], exclude=["atk_percent"])

0.5597652158461518

In [633]:
# Table rows with double crit
baserows = [*([i] for i in substats_keys()), ["crit_rate", "crit_dmg"]]

In [634]:
# Creates an occurrence probability table
# - baserows: for each row, which list of substats to expect (a list of lists)
# - mainstats: a mainstat for column
def run_table(baserows=baserows, mainstats=[]):
  table = []
  for stats in baserows:

    # Row title contains all substats
    row = [" & ".join(stats)]

    # Run the program for each mainstat, excluding repeated with substats
    for exclude in mainstats:
      if exclude in stats:
        row.append("")
      else:
        row.append(run(stats, [exclude]))

    table.append(row)

  return pd.DataFrame(table, columns=["stat(s)", *mainstats])

## Flower

In [635]:
run_table(mainstats=["hp"])

,stat(s),hp
0,hp,
1,atk,0.577642
2,def,0.577642
3,hp_percent,0.432139
4,atk_percent,0.432139
5,def_percent,0.432139
6,energy_recharge,0.432139
7,elemental_mastery,0.432139
8,crit_rate,0.34201
9,crit_dmg,0.34201


## Plume

In [636]:
run_table(mainstats=["atk"])

,stat(s),atk
0,hp,0.577642
1,atk,
2,def,0.577642
3,hp_percent,0.432139
4,atk_percent,0.432139
5,def_percent,0.432139
6,energy_recharge,0.432139
7,elemental_mastery,0.432139
8,crit_rate,0.34201
9,crit_dmg,0.34201


## Sands

In [637]:
run_table(mainstats=["hp_percent", "atk_percent", "def_percent", "energy_recharge", "elemental_mastery"])

,stat(s),hp_percent,atk_percent,def_percent,energy_recharge,elemental_mastery
0,hp,0.559765,0.559765,0.559765,0.559765,0.559765
1,atk,0.559765,0.559765,0.559765,0.559765,0.559765
2,def,0.559765,0.559765,0.559765,0.559765,0.559765
3,hp_percent,,0.416052,0.416052,0.416052,0.416052
4,atk_percent,0.416052,,0.416052,0.416052,0.416052
5,def_percent,0.416052,0.416052,,0.416052,0.416052
6,energy_recharge,0.416052,0.416052,0.416052,,0.416052
7,elemental_mastery,0.416052,0.416052,0.416052,0.416052,
8,crit_rate,0.328248,0.328248,0.328248,0.328248,0.328248
9,crit_dmg,0.328248,0.328248,0.328248,0.328248,0.328248


## Goblet

In [638]:
run_table(mainstats=["hp_percent", "atk_percent", "def_percent", "dmg_bonus", "elemental_mastery"])

,stat(s),hp_percent,atk_percent,def_percent,dmg_bonus,elemental_mastery
0,hp,0.559765,0.559765,0.559765,0.511844,0.559765
1,atk,0.559765,0.559765,0.559765,0.511844,0.559765
2,def,0.559765,0.559765,0.559765,0.511844,0.559765
3,hp_percent,,0.416052,0.416052,0.375215,0.416052
4,atk_percent,0.416052,,0.416052,0.375215,0.416052
5,def_percent,0.416052,0.416052,,0.375215,0.416052
6,energy_recharge,0.416052,0.416052,0.416052,0.375215,0.416052
7,elemental_mastery,0.416052,0.416052,0.416052,0.375215,
8,crit_rate,0.328248,0.328248,0.328248,0.294198,0.328248
9,crit_dmg,0.328248,0.328248,0.328248,0.294198,0.328248


## Circlet

In [639]:
run_table(mainstats=["hp_percent", "atk_percent", "def_percent", "crit_rate", "crit_dmg", "healing_bonus", "elemental_mastery"])

,stat(s),hp_percent,atk_percent,def_percent,crit_rate,crit_dmg,healing_bonus,elemental_mastery
0,hp,0.559765,0.559765,0.559765,0.54895,0.54895,0.511844,0.559765
1,atk,0.559765,0.559765,0.559765,0.54895,0.54895,0.511844,0.559765
2,def,0.559765,0.559765,0.559765,0.54895,0.54895,0.511844,0.559765
3,hp_percent,,0.416052,0.416052,0.406579,0.406579,0.375215,0.416052
4,atk_percent,0.416052,,0.416052,0.406579,0.406579,0.375215,0.416052
5,def_percent,0.416052,0.416052,,0.406579,0.406579,0.375215,0.416052
6,energy_recharge,0.416052,0.416052,0.416052,0.406579,0.406579,0.375215,0.416052
7,elemental_mastery,0.416052,0.416052,0.416052,0.406579,0.406579,0.375215,
8,crit_rate,0.328248,0.328248,0.328248,,0.320256,0.294198,0.328248
9,crit_dmg,0.328248,0.328248,0.328248,0.320256,,0.294198,0.328248
